In [1]:
import numpy as np
import itertools

In [2]:
def kernelDTW(X,Y,sigma):
    '''
    return kernel distance of sequence X and Y
    '''
    if len(X.shape)==1:
        X = np.expand_dims(X, axis=1)
        Y = np.expand_dims(Y, axis=1)
    N = len(X)
    M = len(Y)
    d = np.zeros([N,M])
    xsq = np.sum(X**2,axis=1)
    ysq = np.sum(Y**2,axis=1)
    tD = np.tile(xsq,[M,1]).T + np.tile(ysq,[N,1]) - 2*np.matmul(X,Y.T)
    d = np.exp(-tD/(2*sigma**2))

    # find maximum
    D = np.zeros([N,M])
    D[0,0]=d[0,0]
    for i in range(1,N):
        D[i,0] = d[i,0]+D[i-1,0]
    for j in range(1,M):
        D[0,j] = d[0,j]+D[0,j-1]
    for i in range(1,N):
        for j in range(1,M):
            D[i,j] = d[i,j]+max(D[i-1,j],D[i-1,j-1],D[i,j-1])

    # find path
    path = []
    path.append([N-1,M-1])
    n,m = N-1,M-1
    while((n+m)!=0):
        if (n-1)==-1:
            m=m-1
        elif (m-1)==-1:
            n=n-1
        else:
            argm = np.argmax([D[i-1,j],D[i,j-1],D[i-1,j-1]])
            if argm==0:
                n-=1
            elif argm==1:
                m-=1
            else:
                n-=1
                m-=1
        path.insert(0,[n,m])

    # count kernel
    kernel = 0
    normF = 0
    for i in range(len(path)):
        kernel += d[path[i][0],path[i][1]]*D[path[i][0],path[i][1]]
        normF += D[path[i][0],path[i][1]]
    if normF==0:
        normF=1
    kernel = kernel/normF
    return kernel

def kernelMatrix(X,sigma):
    K = np.zeros([len(X),len(X)])
    for i in range(len(X)):
        for j in range(i,len(X)):
            kernel = kernelDTW(X[i],X[j],sigma)
            K[i,j] = kernel
            K[j,i] = kernel
    return K


In [3]:
def one_hot(a):
    # a = np.array([1, 0, 3])
    b = np.zeros((a.size, a.max()+1))
    b[np.arange(a.size),a] = 1
    return b

def knkmeans(K,k):
    m = len(K)
    label = np.random.randint(k,size=m)
    last= np.zeros_like(label)
    while((label==last).all()):
        u = np.unique(label)
        k = len(u)
        E = one_hot(label)
        E = E/sum(E,0)
        T = np.matmul(E.T,K)
        Z = np.tile(np.diag(np.matmul(T,E)),[m,1]).T - 2*T
        last=label
        label = np.argmin(Z,axis=0)
    return label


In [4]:
def dist_to_center(X,Zci,sigma):
    '''
    count square kernel distance between X and cluster c center
    Zci consist of all sequence segments belongs to cluster ci
    '''
    Nmc = len(Zci)
    first = kernelDTW(X,X,sigma)
    second = np.zeros_like(first)
    for i in range(Nmc):
        second = second + kernelDTW(X,Zci[i],sigma)
    second = second*2/Nmc
    third = np.zeros_like(first)
    for i in range(Nmc):
        for j in range(Nmc):
            third = third + kernelDTW(Zci[i],Zci[j],sigma)
    third = third/(Nmc**2)
    return first - second + third

def min_g(k,X,Zc,sigma):
    '''
    iterate over k clusters to find min g (which class X belongs)
    Zc : sequence segments of different class c
    '''
    g = 0
    minD = dist_to_center(X,Zc[0],sigma)
    for i in range(1,k):
        d2c = dist_to_center(X,Zc[i],sigma)
        if d2c < minD:
            minD = d2c
            g = i
    return g, minD


def Jv(v,J,k,raw,Zc,sigma,nmax):
    '''
    count J(v) for the forwarding step
    v : current frame
    J : DP record
    k : k-means's k
    raw : whole sequence
    Zc : sequence segments of different class c
    nmax : max sequence length
    '''
    imin = max(v-nmax,0)
    minJ = float('inf')
    for i in range(imin,v+1):
        if i==0:
            Jprev = 0
        else:
            Jprev = J[i-1]
        X = raw[i:v+1]
        g, minD = min_g(k,X,Zc,sigma)
        newJ = Jprev + minD
        if newJ < minJ:
            minJ = newJ
            g_star = g
            i_star = i
    return minJ, g_star, i_star

In [5]:
def forward(k,raw,Zc,sigma,nmax):
    '''
    k : k-means's k
    raw : whole sequence
    Zc : sequence segments of different classes c
    nmax : max sequence length
    '''
    n = len(raw)
    J = np.zeros(n)
    g_stars = np.zeros(n)
    i_stars = np.zeros(n)
    for i in range(n):
        J[i],g_stars[i],i_stars[i] = Jv(i,J,k,raw,Zc,sigma,nmax)
    return J, g_stars, i_stars

def backward(k,raw,Zc,sigma,nmin,nmax,g_stars,i_stars):
    '''
    k : k-means's k
    raw : whole sequence
    Zc : sequence segments of different classes c
    g_stars,i_stars : return by forward step
    '''
    n = len(raw)
    cutoff = []
    last = n
    cutoff.append(last)
    for i in range(n):
        s = n-i-1
        if s==0:
            break
        if last-s < nmin: # shorter than min length
            continue
        X_tmp = raw[s:last]
        g ,_ = min_g(k,X_tmp,Zc,sigma)
        if s==i_stars[s] and g==g_stars[s]:
            last = s
            cutoff.insert(0,s) # tail+1 position
        elif last-s > nmax: #exceed max length
            last = s
            cutoff.insert(0,s)
    newZ = []
    head = 0
    for c in cutoff:
        newZ.append(raw[head:c])
        head = c
    return newZ, cutoff

In [6]:
def ACA(k,X,sigma,nmin,nmax):
    '''
    k : k-means's k
    X : N(samples)*F(features) matrix
    sigma : kernel param
    nmax : max sequence length
    '''
    # initial cut => Z, cutoff

    # same length cutoff
    # Z = []
    # cutoff = []
    # inic = int(len(X)/nmax)
    # for i in range(inic):
    #     Z.append(X[i*nmax:(i+1)*nmax])
    #     cutoff.append((i+1)*nmax-1)
    # Z.append(X[inic*nmax:])
    # cutoff.append(len(X)-1)

    # random cutoff
    Z = []
    cutoff = []
    curr = 0
    while(curr<len(X)):
        length = np.random.choice(np.arange(nmin,nmax+1))
        tail = min(len(X),curr+length-1)
        Z.append(X[curr:tail+1])
        cutoff.append(tail)
        curr = tail+1

    # ACA part
    newcut = cutoff
    cutoff = [0]*len(cutoff)
    print(newcut)
    while(cutoff!=newcut):
        cutoff = newcut
        # kmeans count Zc : clustering result of Z
        label = knkmeans(Z,k)
        Zc = []
        for i in range(k):
            Zc.append(list(itertools.compress(Z, label==i)))
        # forward
        J, g_stars, i_stars = forward(k,X,Zc,sigma,nmax)
        # backward
        Z, newcut = backward(k,X,Zc,sigma,nmin,nmax,g_stars,i_stars)
        print(newcut)
    return Z

In [7]:
import scipy.io
X = scipy.io.loadmat('testX.mat')['X'].squeeze().astype(float)
X=X.T
# X = scipy.io.loadmat('testX.mat')['XD'].squeeze().astype(int)

In [8]:
X.shape

(1612, 42)

In [11]:
# X = np.arange(105)#np.dstack([np.arange(105),np.arange(105)]).squeeze()
nmax = 60
nmin = 32
sigma = 10
k=4

Z = ACA(k,X,sigma,nmin,nmax)

[55, 102, 158, 190, 228, 268, 320, 380, 432, 486, 525, 568, 612, 659, 707, 746, 782, 821, 862, 916, 966, 1016, 1060, 1103, 1153, 1209, 1251, 1284, 1343, 1399, 1436, 1474, 1506, 1557, 1595, 1612]


KeyboardInterrupt: 